# Фаза 2 • Неделя 8 • Понедельник
## Нейронные сети
### 🔥 PyTorch

https://pytorch.org/

In [2]:
import torch

torch.manual_seed(42)

# nn - модуль со слоями
from torch import nn

# optim - оптимизаторы
from torch import optim

# TensorDataset – класс датасета, с помощью которого мы укажем DataLoader что у нас за данные
from torch.utils.data import TensorDataset, DataLoader, random_split

# https://github.com/anjandeepsahni/torchutils <- нужно установить
import torchutils as tu
import matplotlib.pyplot as plt

# import mplcyberpunk

# plt.style.use("cyberpunk")
import numpy as np
import pandas as pd

In [4]:
# сделаем dataframe для seaborn
df = pd.read_csv("data/table_dataset.csv")
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,-0.372544,0.149667,-1.829788,-0.710564,0.172706,-0.753473,3.219539,-2.258949,0.689458,0.215635,0.564078,0.347763,1.052765,-2.066468,0.068826,0.0
1,-0.920162,-1.471595,-3.181971,-2.302627,0.078188,0.191089,2.898332,-3.336875,-1.467591,0.047672,-0.734148,-0.166591,0.870139,-0.284713,-0.042125,0.0
2,-0.867346,-0.714984,-2.308342,1.355378,-0.165081,0.253880,2.210411,-2.444333,-0.958174,0.773928,-0.464096,-1.583906,0.319182,-0.338673,-0.512475,0.0
3,-2.593742,-0.488281,-2.770813,-1.514421,-0.074888,0.413238,3.281355,-3.071604,-0.529921,-0.775045,-0.158164,0.334897,-0.409302,-1.176140,0.971471,0.0
4,-0.207870,-0.693828,-2.322182,0.968451,1.672065,-1.305289,0.389738,-2.057347,-2.774857,-1.443170,-1.631638,0.104118,1.234805,1.906438,2.120509,1.0


In [5]:
# разделим призанки от таргета
X, y = df.iloc[:, :-1].values, df.iloc[:, -1].values
n_features = X.shape[1]
print(X.shape, y.shape)
print(n_features)

(100000, 15) (100000,)
15


In [6]:
torch.randn(32, n_features).shape

torch.Size([32, 15])

In [7]:
# зададим модель: на вход будет поступать n_featuers признаков (в нашем случае 15)
# Решаем задачу бинарной классификации, на выходе 1 значение.
model = nn.Sequential(nn.Linear(n_features, 1), nn.Sigmoid())

tu.get_model_summary(model, torch.randn(90, n_features))

Layer   Kernel    Output    Params   FLOPs
0_0     [15, 1]   [90, 1]       16   2,610
1_1           -   [90, 1]        0     360
Total params: 16
Trainable params: 16
Non-trainable params: 0
Total FLOPs: 2,970 / 2.97 KFLOPs
------------------------------------------
Input size (MB): 0.01
Forward/backward pass size (MB): 0.00
Params size (MB): 0.00
Estimated Total Size (MB): 0.01


Фактически мы задали обычную логистическую регрессию (она же обычный нейрон с сигмоидальной функцией активации): 

$$\sigma = \dfrac{1}{1+ \exp^{-\sum_{i=0}^{nfeatures}x_i w_i}}$$

### Подготовка датасета

Будем использовать `TensorDataset`, он подходит нам, потому что мы работаем с табличными данными. Для картинок будем использовать аналогичный специальный класс, об этом позже. 

In [ ]:
# ?TensorDataset

In [8]:
# основной тип данных pytorch - тензор, все данные должны быть в таком формате
dataset = TensorDataset(
    torch.tensor(X, dtype=torch.float), torch.tensor(y, dtype=torch.float)
)

In [9]:
dataset

In [ ]:
# аналог train_test_split для pytorch
train_ds, valid_ds = random_split(dataset, lengths=(0.7, 0.3))

In [ ]:
train_ds

In [ ]:
# DataLoader – загрузчик данных для pytorch,
# он оптимизирует процесс чтения данных и позволяет
# более эффективно использовать ресурсы компьютера
train_loader = DataLoader(train_ds, shuffle=True, batch_size=128)
valid_loader = DataLoader(valid_ds, shuffle=True, batch_size=128)

In [ ]:
# Это обычный генератор, он будет возвращать нам «пакеты» (батчи)
# данных. Мы будем использовать его почти всегда
next(iter(train_loader))

In [ ]:
# Зададим функцию для отрисовки графиков


def plot_loss_metrics(tl: list, vl: list, tm: list, vm: list):
    _, ax = plt.subplots(1, 2, figsize=(14, 5))
    ax[0].plot(tl, label="Train loss")
    ax[0].plot(vl, label="Valid Loss")
    ax[0].legend()
    ax[0].set_title("Loss")
    ax[0].set_ylim((0, max(tl + vl) + 0.1))

    ax[1].plot(tm, label="Train accuracy")
    ax[1].plot(vm, label="Valid accuracy")
    ax[1].legend()
    ax[1].set_title("Accuracy")
    ax[1].set_ylim((0, max(tm + vm) + 0.1))

### Обучение модели

В TensorFlow и pytorch процесс обучения модели программируется вручную, это позволяет более детально следить за процессом и контролировать параметры. Обычно цикл обучения реализуется функцией, но мы попробуем начать с обычного цикла. Процесс итеративный: одна эпоха заканчивается, когда все данные прошли через сеть. 

In [ ]:
for param in model.parameters():
    print(param)

In [ ]:
# задаем оптимизатор – алгоритм градиентного спуска,
# который будет искать решение задачи: минимум функции потерь
optimizer = optim.SGD(model.parameters(), lr=0.005)

Формула бинарной кросс-энтропии: 

$$BCELoss = -\sum_{i}^{N}\big(y_i \log p_i + (1-y_i)\log(1-p_i)\big)$$

In [ ]:
# решаем задачу бинарной классификации: будем минимизировать
# бинарную кросс-энтропию (BCE - Binary Cross Entropy)
criterion = nn.BCELoss()  # nn.MSELoss()

In [ ]:
def fit_model(model: torch.nn.modules.container.Sequential, n_epochs: int) -> tuple:

    # будем сохранять историю обучения модели: значения функции потерь и метрики accuracy
    # эти переменные будут отвечать за хранение значений после вычисления каждой эпохи
    train_losses = []
    valid_losses = []

    train_metric = []
    valid_metric = []

    for epoch in range(n_epochs):
        ####### ОБУЧЕНИЕ ##########
        model.train()  # устанавливаем модель в режим обучения – сейчас будут вычисляться градиенты

        # эти два списка будут хранить значения внутри одной итерации
        train_loss_iter = []
        train_metric_iter = []

        # внутри одной эпохи итерируемся по загрузчику: обычно памяти меньше,
        # чем данных, поэтому сразу взять всю выборку не получится
        for samples, labels in train_loader:

            # получаем предсказания модели, т.е. делаем прямой проход – forward pass
            predictions = model(samples)

            # (128, 1) X (128)
            # на выходе из модели форма predictions (batch_size, 1), а форма labels – (batch_size, )
            # нам нужно убрать одно измерение из predictions, это можно сделать с
            # помощью метода squeeze(-1)
            predictions = predictions.squeeze(-1)

            # считаем значение функции потерь: сравниваем
            # прогнозы модели с истинными значениями классов
            loss = criterion(predictions, labels)

            # обнуляем значения градиентов с предыдущего шага – pytorch этого не делает за нас
            optimizer.zero_grad()
            # делаем обратный проход – backward
            loss.backward()
            # применяем вычисленные градиенты к параметрам
            optimizer.step()

            # запишем loss в список
            train_loss_iter.append(loss.item())

            # вычислим точность: число совпавших классов в предсказании модели и настоящих меток
            accuracy = (torch.round(predictions) == labels).sum() / len(samples)
            train_metric_iter.append(accuracy)

        ###### ВАЛИДАЦИЯ ###########
        # Теперь будем идти по валидационному загрузчику и проверять качество модели на другой выборке
        # переводим модель в режим валидации
        model.eval()

        valid_loss_iter = []
        valid_metric_iter = []

        for samples, labels in valid_loader:

            # получаем предсказания модели, т.е. делаем прямой проход – forward pass
            # делаем это с отключенной функцией вычисления градиентов, нам это ни к чему
            # во время проверки точности модели на валидационной части выборки
            # with torch.inference_mode:
            with torch.inference_mode():
                predictions = model(samples)

            # на выходе из модели форма predictions (batch_size, 1), а форма labels – (batch_size, )
            # нам нужно убрать одно измерение из predictions, это можно сделать с
            # помощью метода squeeze(-1)
            predictions = predictions.squeeze(-1)

            # считаем значение функции потерь: сравниваем прогнозы модели с истинными значениями классов
            loss = criterion(predictions, labels)

            # запишем loss в список, item() забирает только значение из переменной, не тянет за собой техническую информацию
            valid_loss_iter.append(loss.item())

            # вычислим точность: число совпавших классов в предсказании модели и настоящих меток
            accuracy = (torch.round(predictions) == labels).sum() / len(samples)
            valid_metric_iter.append(accuracy)

        # после окончания эпохи запишем все усредненные характеристики в переменные
        train_losses.append(np.mean(train_loss_iter))
        valid_losses.append(np.mean(valid_loss_iter))

        train_metric.append(np.mean(train_metric_iter))
        valid_metric.append(np.mean(valid_metric_iter))

        if epoch % 1 == 0:
            print(
                f"Epoch {epoch} finished: train_loss={train_losses[-1]:.3f}, valid_loss={valid_losses[-1]:.3f}"
            )

    return train_losses, valid_losses, train_metric, valid_metric

In [ ]:
train_losses, valid_losses, train_metric, valid_metric = fit_model(model, 20)

In [ ]:
# Обучение прошло, теперь надо визуализировать лоссы и метрики: только по графикам можно понять, обучилась ли модель
plot_loss_metrics(train_losses, valid_losses, train_metric, valid_metric)

In [ ]:
# Можно посмотреть на значения параметров
# requires_grad означает, что эти параметры участвуют в обучении сети.
# Иногда мы будем их «выключать», т. е. устанавливать это поле в False

for param in model.parameters():
    print(param)

Обычный вопрос: 
> сколько нейронов и слоев необходимо выбирать для решения задачи? 

Исчерпывающего ответа **нет**, можно почитать ответы на [stats.stackexchange.com](https://stats.stackexchange.com/questions/181/how-to-choose-the-number-of-hidden-layers-and-nodes-in-a-feedforward-neural-netw)

In [ ]:
# Зададим другую модель

big_model = nn.Sequential(
    nn.Linear(n_features, 32),
    nn.Dropout(),
    nn.Sigmoid(),
    nn.Linear(32, 16),
    nn.Dropout(),
    nn.Sigmoid(),
    nn.Linear(16, 1),
    nn.Sigmoid(),
)

tu.get_model_summary(big_model, torch.randn(128, n_features))

In [ ]:
optimizer = torch.optim.SGD(big_model.parameters(), lr=0.005)

In [ ]:
tl, vl, tm, vm = fit_model(big_model, 20)

In [ ]:
# Обучение прошло, теперь надо визуализировать лоссы и метрики:
# только по графикам можно понять, обучилась ли модель
plot_loss_metrics(tl, vl, tm, vm)

In [ ]:
# Зададим модель еще сложнее

giant_model = nn.Sequential(
    nn.Linear(n_features, 512),
    nn.Sigmoid(),
    nn.Linear(512, 256),
    nn.Sigmoid(),
    nn.Linear(256, 128),
    nn.Sigmoid(),
    nn.Linear(128, 128),
    nn.Sigmoid(),
    nn.Linear(128, 1),
    nn.Sigmoid(),
)

tu.get_model_summary(giant_model, torch.randn(128, n_features))

In [ ]:
optimizer = torch.optim.SGD(giant_model.parameters(), lr=0.05)
tl, vl, tm, vm = fit_model(giant_model, 20)

In [ ]:
plot_loss_metrics(tl, vl, tm, vm)